In [1]:
from openeye import oechem
import openeye
import oenotebook as oenb
import csv
import os
from pathlib import Path
import pandas as pd
import numpy as np

if not openeye.OEChemIsLicensed():
    raise RunTimeError("Can't find a valid OpenEye license")

### Loading microstates from SAMPL6 files

SAMPL6 microstate definitions are loaded into a dictionary using `load_microstate_dictionary`.

By default it uses from the files at www.github.com/MobleyLab/SAMPL6 , a local copy is stored in the microstates folder.

The states are read in as smiles. For a proper comparison of microstates, explicit hydrogens are added to the molecules using the `add_h` function.



In [2]:
def add_h(mol: oechem.OEMol):
    """Add explicit hydrogens to a molecule"""
    for atom in mol.GetAtoms():
        oechem.OEAddExplicitHydrogens(mol, atom)

def load_microstate_dictionary(sampl_id: str, microstate_folder: str= "./microstates/"):
    """Load a dictionary of microstates as defined by SAMPL from the microstate files.
    
    Parameters
    ----------
    sampl_id - the identifier for the molecule in sampl, e.g. SM12 
    
    Returns
    -------
    dict - key : microstate_id, value : oe_mol
    """

    microstates = dict()

    # Locate the file containing state definitions
    filename = "{}_microstates.csv".format(sampl_id)
    full_path = os.path.join(microstate_folder, filename)
    mypath = Path(full_path)
    if not mypath.is_file():
        raise ValueError("No microstate definitions were found for molecule {}. Check for typos.".format(sampl_id))
    
    # SAMPL6 CSV files are not OpenEye compatible, so instead we read the csv line by line and translate the smiles.
    with open(full_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
        
        for row in csvreader:
            key = row['microstate ID']
            val = oechem.OEGraphMol()
            oechem.OESmilesToMol(val, row['canonical isomeric SMILES'])
            add_h(val)
            microstates[key] = val
            
    return microstates

### Loading results from Epik

This function loads the results for a single SAMPL6 molecule at a single pH function. By default it uses
the `output/microscopic` folder, which contains pKa predictions between 3.0 and 11.0 for states with a population of > 10-5

In [3]:
def load_result(sampl_id:str, pH_str: str, epik_output_folder:str="./output/microscopic/"):
    """Load a single result from Epik files using SAMPL ID, string of the pH with 1 decimal point.
    
    Returns
    -------
    list - OEMols with SD tags containing epik information, every OEMol represents one microstate
    """
    results = list()
        
    
    # Locate the file containing state definitions
    filename = "{}.epik.{}.sdf".format(sampl_id, pH_str)
    full_path = os.path.join(epik_output_folder, sampl_id, filename)
    mypath = Path(full_path)
    if not mypath.is_file():
        raise ValueError("No result file was found for molecule {} at pH {}. Check for typos.".format(sampl_id, pH_str))
    
    ifs = oechem.oemolistream()
    
    if ifs.open(full_path):
        if not oechem.OEIsSDDataFormat(ifs.GetFormat()):
            raise IOError("No SD data")
    
        # Need graphmols for SDF loading
        for mol in ifs.GetOEGraphMols():
            #add_h(mol)
            newmol = oechem.OEGraphMol(mol)
            results.append(newmol)
            # print(newmol.HasSDData())

        ifs.close()
    else:
        raise IOError("Could not read input file.")
    return results

In [4]:
def are_equal_microstates(mol1: oechem.OEGraphMol, mol2: oechem.OEGraphMol, verbose: bool=False):
    """ Check if two supplied OE(Graph)Mol objects have the same molecular microstate present.
    
    If all atoms and bonds are matched in MCSS returns True. Ignores charges and bond order
    to deal with chirality and geometry differences as well as resonance structures
    (which we won't consider as different).
    
    Returns
    -------
    bool    
    """
    # Copy input
    pattern = oechem.OEGraphMol(mol1) 
    target = oechem.OEGraphMol(mol2)
    
    # Atoms are equal if they have same atomic number (so explicit Hydrogens are needed as well for a match)
    atomexpr = oechem.OEExprOpts_AtomicNumber&oechem.OEExprOpts_HCount
     # single or double bonds are considered identical (resonance,chirality fix)
    bondexpr = oechem.OEExprOpts_EqSingleDouble
    # create maximum common substructure object
#     mcss = oechem.OEMCSSearch(pattern, atomexpr, bondexpr, oechem.OEMCSType_Exhaustive)
    mcss = oechem.OEMCSSearch(pattern, atomexpr, bondexpr, oechem.OEMCSType_Approximate)

    
    # set scoring function
    mcss.SetMCSFunc(oechem.OEMCSMaxAtomsCompleteCycles())
    # ignore matches smaller than 6 atoms
    mcss.SetMinAtoms(6)
    unique = True


    # loop over matches
    count = 0
    match = oechem.OEMol()
    for i, match in enumerate(mcss.Match(target, unique)):
        count = i + 1
        if verbose:
            print("Match %d:" % (count))
    
            print( "Num atoms in match %d" % match.NumAtoms())
            print( "Num atoms in mol1 %d" % pattern.NumAtoms())
            print( "Num atoms in mol2 %d" % target.NumAtoms())


    # check if there is only single match
    if (count > 1):
        if verbose: print("Warning! There are multiple matches.")
    elif count == 0:
        if verbose: print("No match")
        
    m_num = match.NumAtoms()
    p_num = pattern.NumAtoms()
    t_num = target.NumAtoms()
        
        
    return m_num == p_num == t_num


def match_subset(pattern: oechem.OEGraphMol, target:oechem.OEGraphMol):
    """Check if target is a subset of pattern."""
    # Atoms are equal if they have same atomic number (so explicit Hydrogens are needed as well for a match)
    atomexpr = oechem.OEExprOpts_AtomicNumber
     # single or double bonds are considered identical (resonance,chirality fix)
    bondexpr = oechem.OEExprOpts_EqSingleDouble
    ss = oechem.OESubSearch(pattern, atomexpr, bondexpr )
    oechem.OEPrepareSearch(target, ss)

    return ss.SingleMatch(target)

def match_microstates(mol1, mol2):
    """If both states are contained in each other, they're the same."""
    return match_subset(mol1, mol2) and match_subset(mol2, mol1)

# Testing


# electrons shifted for benzene
mol1 = oechem.OEMol()
oechem.OESmilesToMol(mol1, "C1=C[CH-]=CC=[CH+]1")
add_h(mol1)

# regular benzene
mol2 = oechem.OEMol()
oechem.OESmilesToMol(mol2, "c1ccccc1")
add_h(mol2)

# protonated benzene
mol3 = oechem.OEMol()
oechem.OESmilesToMol(mol3, "C1=C[CH2-]=CC=[C+]1")
add_h(mol3)


if not match_microstates(mol1, mol2,):
    raise RuntimeError("These should be resonance structures")

if not match_microstates(mol2, mol1, ):
    raise RuntimeError("Order should not matter.")

if match_microstates(mol1, mol3, ):
    raise RuntimeError("These should not be equal states.")

if match_microstates(mol2, mol3, ):
    raise RuntimeError("These should not be equal states either.")

In [5]:
def map_result(microstates: dict, results: list):
    """Match states from results at one pH to microstates."""
    matches, unmatched = [], []
    for index, result_state in enumerate(results):
        mapping = {"Epik_Microstate": "{:d}".format(index),
                   "Epik_Molecule": result_state, "SMILES" : oechem.OECreateCanSmiString(result_state)}
        matched = False
        for microstate_id, microstate in microstates.items():
            newmatch = match_microstates(result_state, microstate)
            if newmatch and matched:
#                 print(index, microstate_id, "matched more than once!")
                duplicate_mapping = dict(mapping)
                duplicate_mapping["SAMPL6_Microstate_ID"] = microstate_id
                duplicate_mapping["SAMPL6_Molecule"] = microstate                
                matches.append(duplicate_mapping)
            elif newmatch:
                matched = True
#                 print(index, microstate_id, "matched!")
                mapping["SAMPL6_Microstate_ID"] = microstate_id
                mapping["SAMPL6_Molecule"] = microstate
                matches.append(mapping)
                
        if matched == False:
#             print("Could not match: ", index)
            unmatched.append(mapping)
    return matches, unmatched

In [6]:
unmatched_states = pd.DataFrame(columns=["Epik_Microstate", "Epik_Molecule", "pH", "SAMPL6_ID", "SAMPL6_Microstate_ID", "SAMPL6_Molecule"])
matched_states = pd.DataFrame(columns=["Epik_Microstate", "Epik_Molecule", "pH", "SAMPL6_ID", "SAMPL6_Microstate_ID", "SAMPL6_Molecule"])

for molecule in range(24):
    SAMPL_ID = "SM{:02d}".format(molecule+1)
    print(SAMPL_ID)
    microstates = load_microstate_dictionary(SAMPL_ID)
    for pH in np.linspace(3.0, 11.0, 81):
        if pH % 3.0 == 0:
            print(pH)
        pH_str = '{:.1f}'.format(pH)
        results = load_result(SAMPL_ID, pH_str)        
        matched, unmatched = map_result(microstates, results)
        for um in unmatched:
            um['pH'] = pH_str
            um['SAMPL6_ID'] = SAMPL_ID
            unmatched_states = unmatched_states.append(um, ignore_index=True)
        for m in matched:
            m['pH'] = pH_str
            m['SAMPL6_ID'] = SAMPL_ID
            matched_states = matched_states.append(m, ignore_index=True)

SM01
3.0
6.0
9.0
SM02
3.0
6.0
9.0
SM03
3.0
6.0
9.0
SM04
3.0
6.0
9.0
SM05
3.0
6.0
9.0
SM06
3.0
6.0
9.0
SM07
3.0
6.0
9.0
SM08
3.0
6.0
9.0
SM09
3.0
6.0
9.0
SM10
3.0
6.0
9.0
SM11
3.0
6.0
9.0
SM12
3.0
6.0
9.0
SM13
3.0
6.0
9.0
SM14
3.0
6.0
9.0
SM15
3.0
6.0
9.0
SM16
3.0
6.0
9.0
SM17
3.0
6.0
9.0
SM18
3.0
6.0
9.0
SM19
3.0
6.0
9.0
SM20
3.0
6.0
9.0
SM21
3.0
6.0
9.0
SM22
3.0
6.0
9.0
SM23
3.0
6.0
9.0
SM24
3.0
6.0
9.0


In [7]:
#unmatched_states_filtered = unmatched_states.drop(["pH", "SAMPL6_Microstate_ID", "SAMPL6_Molecule"], 1).drop_duplicates("SMILES")

sampl_duplicated = matched_states.duplicated(["pH", "SAMPL6_ID", "SAMPL6_Microstate_ID"], keep=False)
epik_duplicated = matched_states.duplicated(["pH", "SAMPL6_ID", "Epik_Microstate"], keep=False)

In [21]:
# oenb.render_dataframe(matched_states[sampl_duplicated], mol_col="Epik_Molecule")

In [9]:
# oenb.render_dataframe(matched_states[matched_states["SAMPL6_ID"]== "SM02"], mol_col="Epik_Molecule")

In [15]:
matched_states['Log_P'] = matched_states.apply(lambda row: np.float(oechem.OEGetSDData(row['Epik_Molecule'],"r_epik_State_Penalty"))/ (-298.15 * 1.9872036e-3), axis=1)
matched_states.to_csv(open("microscopic_to_sampl6_mapping.csv", "w"), columns=["SAMPL6_ID", "pH", "Epik_Microstate", "SAMPL6_Microstate_ID", "Log_P"])

In [14]:
oenb.render_dataframe(unmatched_states, mol_col="Epik_Molecule")

,Epik_Microstate,Epik_Molecule,pH,SAMPL6_ID,SAMPL6_Microstate_ID,SAMPL6_Molecule


In [45]:
# shorthand
final_df = pd.DataFrame(columns = ["SAMPL6_ID", "SAMPL6_Microstate_ID", "pH", "Log_P"], )
ms = matched_states

def register_result(molecule_id, output_df, pH_array=None):
    """Register results in the final dataframe"""

    microstates = load_microstate_dictionary(molecule_id)
    for microstate in microstates.keys():
        if not any(ms["SAMPL6_Microstate_ID"]== microstate):
            continue
            
        if pH_array is None:
            pH_array = np.linspace(2.0, 12.0, 101)
        for pH in pH_array:
            pH_str = '{:.1f}'.format(pH)
            new_entry = {"SAMPL6_ID": molecule_id, "SAMPL6_Microstate_ID": microstate, "pH" : float(pH) , "Log_P" : ""}
            data = ms.loc[(ms['SAMPL6_ID'] == SAMPL_ID) & (ms['SAMPL6_Microstate_ID'] == microstate) & (ms['pH'] == pH_str) ] 
            
            if len(data) > 0:               
                # In case duplicates, pick num 1                
                row = data.iloc[0]
                new_entry["Log_P"] = row["Log_P"]

            output_df = output_df.append(new_entry, ignore_index=True)
    return output_df

for molecule in range(24):
    SAMPL_ID = "SM{:02d}".format(molecule+1)
    final_df = register_result(SAMPL_ID, final_df)
    
    
final_df

,SAMPL6_ID,SAMPL6_Microstate_ID,pH,Log_P
0,SM01,SM01_micro004,2.0,
1,SM01,SM01_micro004,2.1,
2,SM01,SM01_micro004,2.2,
3,SM01,SM01_micro004,2.3,
4,SM01,SM01_micro004,2.4,
5,SM01,SM01_micro004,2.5,
6,SM01,SM01_micro004,2.6,
7,SM01,SM01_micro004,2.7,
8,SM01,SM01_micro004,2.8,
9,SM01,SM01_micro004,2.9,


In [47]:
typeII = final_df.pivot(values="Log_P", index="SAMPL6_Microstate_ID", columns="pH")

In [50]:
typeII.to_csv(open("typeII-raw-microscopic.csv", "w"))